In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision
from torchvision import datasets, models, transforms

import os

In [ ]:
from PIL import Image
data_transforms = transforms.Compose([
                  transforms.Resize(224),
                  transforms.ToTensor(),
                  transforms.Normalize([0.83228178, 0.81077838, 0.80394799], [0.12131979, 0.1444876,  0.15186805])
])

is_use_cuda = torch.cuda.is_available()
if is_use_cuda:
    torch.cuda.set_device(3)

In [ ]:
class MyVGG16(nn.Module):
    def __init__(self, vgg_model):
        super(MyVGG16, self).__init__()
        for name, model in vgg_model._modules.items():
            if 'features' == name:
                self.features = copy.deepcopy(model)
            elif 'classifier' == name:
                self.classifier = nn.Sequential()
                for name_t, model_t in model._modules.items():
                    if '6' != name_t:
                        self.classifier.add_module(name_t, model_t)
                
        self.landmarks = nn.Linear(4096, 8)
        self.visible   = nn.Linear(4096, 12)
        
    def forward(self, x):
        ft  = self.features(x)
        ft  = ft.view(ft.size(0), -1) 
        ft  = self.classifier(ft)
        lm  = self.landmarks(ft)
        vb  = self.visible(ft)
        vb  = vb.view(-1, 3)
        return lm, vb
    
class MyVGG16_DeepFashion(nn.Module):
    def __init__(self, landmarks_model, vgg_model):
        super(MyVGG16_DeepFashion, self).__init__()
        self.landmarks    = landmarks_model
        
        self.fc6_local    = nn.Linear(2048*3*3, 2048)        # Notice
        
        stage5_list = []
        for name, module in vgg_model._modules.items():
            if 'features' == name:
                for name_t, module_t in module._modules.items():
                    if int(name_t) >= 19:
                        stage5_list.append(copy.deepcopy(module_t))
        self.stage5_global = nn.Sequential(*stage5_list)
        self.fc6_global    = nn.Linear(25088, 4096)
        
        self.category  = nn.Linear(6144, 5)                # Notice
        self.attribute = nn.Linear(6144, 926)
        
    def forward(self, x1):
        out_feats_l = []
        out_categorys = []
        out_attributes = []
        out_landmark = []
        out_visible_l =[]
        for x in [x1]:
            landmarks_out = []
            for name, model in self.landmarks._modules.items():
                if 'features' == name:
                    for name_t, model_t in model._modules.items():
                        if '19' == name_t:
                            landmarks_out.append(x)
                        x = model_t(x)
                elif 'classifier' == name:
                    x = x.view(x.size(0), -1) 
                    x = model(x)
                elif 'landmarks' == name:
                    out1 = model(x)
                    landmarks_out.append(out1)
                elif 'visible' == name:
                    out2 = model(x)
                    landmarks_out.append(out2)
                else:
                    x = model(x)
            #print(landmarks_out)
            out_landmarks  = np.clip(landmarks_out[1].data.cpu().numpy().reshape(-1, 4, 2), 0.0001, 0.9999)
            out_visible = landmarks_out[2].view(-1, 4, 3).data.cpu().max(2)[1].numpy()
            out_landmarks[out_visible != 0, :] = 0.

            feat_map_x_low = (out_landmarks[:, :, 0] * 27 - 3).astype(int)
            feat_map_x_low[feat_map_x_low < 0] = 0
            feat_map_x_low[feat_map_x_low >= 23] = 22
            feat_map_y_low = (out_landmarks[:, :, 1] * 27 - 3).astype(int)
            feat_map_y_low[feat_map_y_low < 0] = 0
            feat_map_y_low[feat_map_y_low >= 23] = 22

            batch_feats = []
            for i in range(len(feat_map_x_low)):
                tmp_local_feats = []
                for j in range(len(feat_map_x_low[0])):
                    tmp_local_feats.append(landmarks_out[0][i, :, feat_map_x_low[i, j]:feat_map_x_low[i, j]+6, feat_map_y_low[i, j]:feat_map_y_low[i, j]+6].unsqueeze(0))
                local_feats = torch.cat(tmp_local_feats, dim=1)
                batch_feats.append(local_feats)
            batch_feats = torch.cat(batch_feats, dim=0)
            batch_feats = F.max_pool2d(batch_feats, 2)
            batch_feats = batch_feats.view(batch_feats.size(0), -1)
            batch_feats = self.fc6_local(batch_feats)

            global_feats = self.stage5_global(landmarks_out[0])
            global_feats = global_feats.view(global_feats.size(0), -1)
            global_feats = self.fc6_global(global_feats)

            out_feats = torch.cat([global_feats, batch_feats], dim=1)
            out_feats_l.append(out_feats)
            out_categorys.append(self.category(out_feats))
            out_attributes.append(self.attribute(out_feats).view(-1, 2))
            out_landmark.append(landmarks_out[1])
            out_visible_l.append(landmarks_out[2].view(-1, 3))
        
        out_categorys = torch.cat(out_categorys, dim=0)
        out_attributes = torch.cat(out_attributes, dim=0)
        out_landmark = torch.cat(out_landmark, dim=0)
        out_visible_l = torch.cat(out_visible_l, dim=0)
        
        # categorys, attributes, landmarks, visible
        return out_feats_l, out_categorys, out_attributes, out_landmark, out_visible_l

In [ ]:
class MyResNet101(nn.Module):
    def __init__(self, resnet_model):
        super(MyResNet101, self).__init__()
        self.features = nn.Sequential()
        for name, model in resnet_model._modules.items():
            if name not in ['avgpool', 'fc']:
                self.features.add_module(name, copy.deepcopy(model))
                
        self.features.add_module('out_maxpool', nn.MaxPool2d(2, 2))
        self.classifier = nn.Sequential(
                          nn.Linear(18432, 4096),
                          nn.ReLU(True),
                          nn.Dropout(),
                          nn.Linear(4096, 4096),
                          nn.ReLU(True),
                          nn.Dropout()
        )
        self.landmarks = nn.Linear(4096, 8)
        self.visible   = nn.Linear(4096, 12)
        
    def forward(self, x):
        ft  = self.features(x)
        ft  = ft.view(ft.size(0), -1)
        ft  = self.classifier(ft)
        lm  = self.landmarks(ft)
        vb  = self.visible(ft)
        vb  = vb.view(-1, 3)
        return lm, vb
    
class MyResNet101_DeepFashion(nn.Module):
    def __init__(self, landmarks_model, resnet_model):
        super(MyResNet101_DeepFashion, self).__init__()
        self.landmarks    = landmarks_model
        
        self.fc6_local    = nn.Linear(1024*4*3*3, 2048)        # Notice
        
        self.stage5_global = copy.deepcopy(resnet_model.layer4)
        self.stage5_global.add_module('out_maxpool', nn.MaxPool2d(2, 2))
        self.fc6_global    = nn.Linear(18432, 4096)
        
        self.category  = nn.Linear(6144, 5)                # Notice
        self.attribute = nn.Linear(6144, 926)
        
    def forward(self, x1):
        out_feats_l = []
        out_categorys = []
        out_attributes = []
        out_landmark = []
        out_visible_l =[]
        for x in [x1]:
            landmarks_out = []
            for name, model in self.landmarks._modules.items():
                if 'features' == name:
                    for name_t, model_t in model._modules.items():
                        if 'layer4' == name_t:
                            landmarks_out.append(x)
                        x = model_t(x)
                elif 'classifier' == name:
                    x = x.view(x.size(0), -1) 
                    x = model(x)
                elif 'landmarks' == name:
                    out1 = model(x)
                    landmarks_out.append(out1)
                elif 'visible' == name:
                    out2 = model(x)
                    landmarks_out.append(out2)
                else:
                    x = model(x)
            #print(landmarks_out)
            out_landmarks  = np.clip(landmarks_out[1].data.cpu().numpy().reshape(-1, 4, 2), 0.0001, 0.9999)
            out_visible = landmarks_out[2].view(-1, 4, 3).data.cpu().max(2)[1].numpy()
            out_landmarks[out_visible != 0, :] = 0.

            feat_map_x_low = (out_landmarks[:, :, 0] * 13 - 3).astype(int)
            feat_map_x_low[feat_map_x_low < 0] = 0
            feat_map_x_low[feat_map_x_low >= 9] = 8
            feat_map_y_low = (out_landmarks[:, :, 1] * 13 - 3).astype(int)
            feat_map_y_low[feat_map_y_low < 0] = 0
            feat_map_y_low[feat_map_y_low >= 9] = 8

            batch_feats = []
            for i in range(len(feat_map_x_low)):
                tmp_local_feats = []
                for j in range(len(feat_map_x_low[0])):
                    tmp_local_feats.append(landmarks_out[0][i, :, feat_map_x_low[i, j]:feat_map_x_low[i, j]+6, feat_map_y_low[i, j]:feat_map_y_low[i, j]+6].unsqueeze(0))
                local_feats = torch.cat(tmp_local_feats, dim=1)
                batch_feats.append(local_feats)
            batch_feats = torch.cat(batch_feats, dim=0)
            batch_feats = F.max_pool2d(batch_feats, 2)
            batch_feats = batch_feats.view(batch_feats.size(0), -1)
            batch_feats = self.fc6_local(batch_feats)

            global_feats = self.stage5_global(landmarks_out[0])
            global_feats = global_feats.view(global_feats.size(0), -1)
            global_feats = self.fc6_global(global_feats)

            out_feats = torch.cat([global_feats, batch_feats], dim=1)
            out_feats_l.append(out_feats)
            out_categorys.append(self.category(out_feats))
            out_attributes.append(self.attribute(out_feats).view(-1, 2))
            out_landmark.append(landmarks_out[1])
            out_visible_l.append(landmarks_out[2].view(-1, 3))
        
        out_categorys = torch.cat(out_categorys, dim=0)
        out_attributes = torch.cat(out_attributes, dim=0)
        out_landmark = torch.cat(out_landmark, dim=0)
        out_visible_l = torch.cat(out_visible_l, dim=0)
        
        # categorys, attributes, landmarks, visible
        return out_feats_l, out_categorys, out_attributes, out_landmark, out_visible_l

In [ ]:
my_model = torch.load('./checkpoint/DeepFashion_ResNet101_all_4points_stage2.ckpt', map_location=lambda storage, loc: storage.cuda(3))['state_dict']
if is_use_cuda:
    my_model = my_model.cuda()
my_model.eval()

In [ ]:
import numpy as np
DATA_ROOT = '/data/luyi/task_three/data/DeepFashion'
QUERY_FILE   = 'four_points_landmarks_query.txt'
GALLERY_FILE = 'four_points_landmarks_gallery.txt'

for index, file in [('query', QUERY_FILE), ('gallery', GALLERY_FILE)]:
    with open(os.path.join(DATA_ROOT, 'Anno', file), 'r', encoding='utf-8') as fd:
        storage_np = np.zeros((len(fd.readlines()), 6145), dtype=np.float64)
        count = 0
        fd.seek(0, 0)
        for _line in fd.readlines():
            infos = _line.replace('\n', '').split()
            item_id = infos[-1]
            q_id    = int(item_id)
            
            file_name = os.path.join(DATA_ROOT, infos[0])
            img       = Image.open(file_name)
            img_tensor = data_transforms(img).unsqueeze(0)
            if is_use_cuda:
                img_tensor = img_tensor.cuda()
                img_tensor = Variable(img_tensor, volatile=True)
            ft = my_model(img_tensor)[0][0].data.cpu()
            ft = ft / ft.norm(2, 1, True)
            #print(ft)
            ft = ft.numpy()
            storage_np[count][0]  = q_id
            storage_np[count][1:] = ft
            count += 1
        np.save(os.path.join(DATA_ROOT, 'Eval', index+'_4points_stage2.npy'), storage_np)